<a href="https://colab.research.google.com/github/R-Madhuram/DartmouthCapstone_CrudeOilPricePrediction/blob/main/0_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Model

In this notebook we would be building the Baseline model that would act as a benck mark/reference models for all other more poshisticated models we would build. 

This model comprises of taking the mean of the train and test data series and compare it witht he actual data point to calculate the error metric(s). 

We have chosen MSE and RMSE (Mean Squared error and Root Mean Squared Error) as the metric to measure error as it resilient to outliers and can handle fluctuating data points well. This might be useful for the dataset we have chosen ans oil price volatality is a known even that would be dispered in the time series dataset. 

# Libraries

In [ ]:
# Mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='whitegrid')

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet,Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
import io

import itertools


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Load Data

In [ ]:
# load data
us_cushing = pd.read_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/data/USCushing.csv')
eu_brent = pd.read_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/data/EuropeanBrent.csv')

## Cushing Data

In [ ]:
# Read data 
us_cushing.head().append(us_cushing.tail())

,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
0,"Jan 02, 1986",25.56
1,"Jan 03, 1986",26.00
2,"Jan 06, 1986",26.53
3,"Jan 07, 1986",25.85
4,"Jan 08, 1986",25.87
9357,"Feb 22, 2023",73.95
9358,"Feb 23, 2023",75.26
9359,"Feb 24, 2023",76.19
9360,"Feb 27, 2023",75.57
9361,NaN,NaN


## Brent Data

In [ ]:
# Read data
eu_brent.head().append(eu_brent.tail())

,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,"May 20, 1987",18.63
1,"May 21, 1987",18.45
2,"May 22, 1987",18.55
3,"May 25, 1987",18.60
4,"May 26, 1987",18.63
9077,"Feb 22, 2023",79.55
9078,"Feb 23, 2023",82.08
9079,"Feb 24, 2023",82.31
9080,"Feb 27, 2023",82.31
9081,NaN,NaN


# Creating dataframe with date as index and changing date as date-time object

## Cushing Data

In [ ]:
# Convert the date to dateimte object 
us_cushing['Date'] = pd.to_datetime(us_cushing['Date'], format = '%b %d, %Y')

In [ ]:
print(us_cushing['Date'].dtype)

datetime64[ns]


In [ ]:
# rename price column
us_cushing.rename(columns={'Cushing, OK WTI Spot Price FOB (Dollars per Barrel)':'price'},inplace=True)

In [ ]:
cushing_df = us_cushing

In [ ]:
# dropping the last NaN row in the dataframe
cushing_df = cushing_df[:-1]
cushing_df

,Date,price
0,1986-01-02,25.56
1,1986-01-03,26.00
2,1986-01-06,26.53
3,1986-01-07,25.85
4,1986-01-08,25.87
...,...,...
9356,2023-02-21,76.28
9357,2023-02-22,73.95
9358,2023-02-23,75.26
9359,2023-02-24,76.19


In [ ]:
# Change the index of the Df as date index
cushing_df = cushing_df.set_index('Date')

In [ ]:
# setting the datetime index to have a period of DAYS
cushing_df.index = pd.DatetimeIndex(cushing_df.index).to_period('D')
cushing_df

,price
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87
...,...
2023-02-21,76.28
2023-02-22,73.95
2023-02-23,75.26


In [ ]:
# split the data in to the training data and the testing data. The testing data will be used to run the forecast and check the efficiency of the different
# models.
cushing_y_hist = cushing_df.loc['1986':'2021','price'] 
cushing_y_future = cushing_df.loc['2022':, 'price']
cushing_y_hist, cushing_y_future

(Date
 1986-01-02    25.56
 1986-01-03    26.00
 1986-01-06    26.53
 1986-01-07    25.85
 1986-01-08    25.87
               ...  
 2021-12-27    75.49
 2021-12-28    76.01
 2021-12-29    76.58
 2021-12-30    76.83
 2021-12-31    75.33
 Freq: D, Name: price, Length: 9072, dtype: float64,
 Date
 2022-01-03    75.99
 2022-01-04    77.00
 2022-01-05    77.83
 2022-01-06    79.47
 2022-01-07    79.00
               ...  
 2023-02-21    76.28
 2023-02-22    73.95
 2023-02-23    75.26
 2023-02-24    76.19
 2023-02-27    75.57
 Freq: D, Name: price, Length: 289, dtype: float64)

## Brent Data

In [ ]:
# Convert the date to datetime object 
eu_brent['Date'] = pd.to_datetime(eu_brent['Date'], format = '%b %d, %Y')

In [ ]:
print(eu_brent['Date'].dtype)

datetime64[ns]


In [ ]:
# rename price column
eu_brent.rename(columns={'Europe Brent Spot Price FOB (Dollars per Barrel)':'price'},inplace=True)
eu_brent

,Date,price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63
...,...,...
9077,2023-02-22,79.55
9078,2023-02-23,82.08
9079,2023-02-24,82.31
9080,2023-02-27,82.31


In [ ]:
# dropping the last NaN row in the dataframe
brent_df = eu_brent
brent_df = brent_df[:-1]

In [ ]:
# Change the index of the Df as date index
brent_df = brent_df.set_index('Date')

In [ ]:
# setting the datetime index to have a period of DAYS
brent_df.index = pd.DatetimeIndex(brent_df.index).to_period('D')
brent_df

,price
Date,
1987-05-20,18.63
1987-05-21,18.45
1987-05-22,18.55
1987-05-25,18.60
1987-05-26,18.63
...,...
2023-02-21,82.14
2023-02-22,79.55
2023-02-23,82.08


In [ ]:
# split the data in to the training data and the testing data. The testing data will be used to run the forecast and check the efficiency of the different
# models.
brent_y_hist = brent_df.loc['1986':'2021','price'] 
brent_y_future = brent_df.loc['2022':, 'price']
brent_y_hist, brent_y_future

(Date
 1987-05-20    18.63
 1987-05-21    18.45
 1987-05-22    18.55
 1987-05-25    18.60
 1987-05-26    18.63
               ...  
 2021-12-23    76.26
 2021-12-24    75.24
 2021-12-29    78.63
 2021-12-30    78.61
 2021-12-31    77.24
 Freq: D, Name: price, Length: 8789, dtype: float64,
 Date
 2022-01-03    78.25
 2022-01-04    79.39
 2022-01-05    80.60
 2022-01-06    81.99
 2022-01-07    82.28
               ...  
 2023-02-21    82.14
 2023-02-22    79.55
 2023-02-23    82.08
 2023-02-24    82.31
 2023-02-27    82.31
 Freq: D, Name: price, Length: 292, dtype: float64)

# Baseline Results

## Cushing Data

In [ ]:
# The model metrics are computed against a baseline value of the mean of the data. For the training data this is the training period mean, 
# for the test data this is the test period mean. Both the root mean square error and mean square error are computed.

# compute the baseline model metrics for the cushing data.
baseline_metrics_df = {}
baseline_metrics_df['cushing_train_rmse'] = mean_squared_error(cushing_y_hist,np.repeat(cushing_y_hist.mean(),len(cushing_y_hist)),squared=False)
baseline_metrics_df['cushing_test_rmse'] = mean_squared_error(cushing_y_future,np.repeat(cushing_y_future.mean(),len(cushing_y_future)),squared=False)

baseline_metrics_df['cushing_train_mse'] = mean_squared_error(cushing_y_hist,np.repeat(cushing_y_hist.mean(),len(cushing_y_hist)),squared=True)
baseline_metrics_df['cushing_test_mse'] = mean_squared_error(cushing_y_future,np.repeat(cushing_y_future.mean(),len(cushing_y_future)),squared=True)

In [ ]:
# convert the baseline model metrics for the cushing data to a dataframe
cushing_baseline_df = cushing_y_future.to_frame()
cushing_baseline_df['baseline_price'] = np.repeat(cushing_y_future.mean(),len(cushing_y_future))
cushing_baseline_df

,price,baseline_price
Date,,
2022-01-03,75.99,92.615986
2022-01-04,77.00,92.615986
2022-01-05,77.83,92.615986
2022-01-06,79.47,92.615986
2022-01-07,79.00,92.615986
...,...,...
2023-02-21,76.28,92.615986
2023-02-22,73.95,92.615986
2023-02-23,75.26,92.615986


In [ ]:
# export the baseline predictions as a dataframe
cushing_baseline_df.to_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/baseline/cushing_baseline_predictions.csv')

## Brent Data

In [ ]:
# compute the baseline model metrics for the brent data.
baseline_metrics_df['brent_train_rmse'] = mean_squared_error(brent_y_hist,np.repeat(brent_y_hist.mean(),len(brent_y_hist)),squared=False)
baseline_metrics_df['brent_test_rmse'] = mean_squared_error(brent_y_future,np.repeat(brent_y_future.mean(),len(brent_y_future)),squared=False)

baseline_metrics_df['brent_train_mse'] = mean_squared_error(brent_y_hist,np.repeat(brent_y_hist.mean(),len(brent_y_hist)),squared=True)
baseline_metrics_df['brent_test_mse'] = mean_squared_error(brent_y_future,np.repeat(brent_y_future.mean(),len(brent_y_future)),squared=True)

In [ ]:
# convert the baseline model metrics for the brent data to a dataframe
brent_baseline_df = brent_y_future.to_frame()
brent_baseline_df['baseline_price'] = np.repeat(brent_y_future.mean(),len(brent_y_future))
brent_baseline_df

,price,baseline_price
Date,,
2022-01-03,78.25,98.41363
2022-01-04,79.39,98.41363
2022-01-05,80.60,98.41363
2022-01-06,81.99,98.41363
2022-01-07,82.28,98.41363
...,...,...
2023-02-21,82.14,98.41363
2023-02-22,79.55,98.41363
2023-02-23,82.08,98.41363


In [ ]:
# export the baseline predictions as a dataframe
brent_y_future.to_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/baseline/brent_baseline_predictions.csv')

# Results Dataframe

In [ ]:
baseline_metrics_df

{'cushing_train_rmse': 28.77196477838174,
 'cushing_test_rmse': 13.054355852605367,
 'cushing_train_mse': 827.8259572084394,
 'cushing_test_mse': 170.416206726452,
 'brent_train_rmse': 32.020256038619884,
 'brent_test_rmse': 14.01907191054616,
 'brent_train_mse': 1025.2967967787731,
 'brent_test_mse': 196.53437723306433}

In [ ]:
pd.DataFrame.from_dict(baseline_metrics_df,orient='index').to_csv('/content/drive/MyDrive/PCDS_OilPricePrediction/forecasting_products/baseline/baseline_metrics.csv')

Hence, we have bult the baseline model with the above printed values that would act as the benchmark for all our future models if we accept the model based on its learning capacity. 